In [2]:
# Load GPT model
%run gpt-model.ipynb

/Users/diego/Documents/Projects/build-llm/.venv/lib/python3.10/site-packages/torch/nn/modules/transformer.py:20: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  device: torch.device = torch.device(torch._C._get_default_device()),  # torch.device('cpu'),


tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])
 Att Trader willpower Carn
 Tokyo uber apartments targets
Mean:
  tensor([-0.3596, -0.2606])
Variance :
  tensor([0.2015, 0.2673])
Norm. Mean:
  tensor([    -0.0000,      0.0000], grad_fn=<MeanBackward1>)
Norm. Variance :
  tensor([1.0000, 1.0000], grad_fn=<VarBackward0>)
tensor([[0.2685, 0.7413],
        [0.2738, 0.7564],
        [0.2668, 0.7366],
        [0.2618, 0.7218],
        [0.2712, 0.7495]], grad_fn=<MmBackward0>)
tensor([[-0.4927, -0.0791],
        [-0.4938, -0.0806],
        [-0.4924, -0.0851],
        [-0.4923, -0.0819],
        [-0.4928, -0.0853]], grad_fn=<MmBackward0>)
Self Attention V2 output: 
 tensor([[-0.4927, -0.0791],
        [-0.4938, -0.0806],
        [-0.4924, -0.0851],
        [-0.4923, -0.0819],
        [-0.4928, -0.0853]], grad_fn=<MmBackward0>)
Self Attention V1 output: 
 tensor([[-0.4927, -0.0791],
        [-0.4938, -0.0806],
        [-0.4924, -0.0851],
        [-0.4923, -0.0819],
        

# Generating Text

Define a functionality that allows encoding and decoding text

In [3]:
import tiktoken
import torch

def textToTokenIds(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    return torch.tensor(encoded).unsqueeze(0)

def tokenIdsToText(tokens, tokenizer):
    formatted = tokens.squeeze(0).tolist()
    return tokenizer.decode(formatted)


In [4]:
# Test functionality to encode and decode text.

tokenizer = tiktoken.get_encoding("gpt2")

testText = "Hello world <|endoftext|>"
print("Original:", testText)

encoded = textToTokenIds(testText, tokenizer)
print("Encoded: ", encoded)

decoded = tokenIdsToText(encoded, tokenizer)
print("Decoded: ", decoded)


del tokenizer, testText, encoded, decoded
# generateText(model, textToTokenIds(testText, tokenizer), 10, cfg["contextSize"] )

Original: Hello world <|endoftext|>
Encoded:  tensor([[15496,   995,   220, 50256]])
Decoded:  Hello world <|endoftext|>


Test functionality to run GPT model to generate text

In [5]:
import torch

tokenizer = tiktoken.get_encoding("gpt2")
maxNewTokens = 10
testText = "Hello world"

# Run model
encodedOutput = generateText(model, textToTokenIds(testText, tokenizer), maxNewTokens, cfg["contextLength"] )
print("Encoded output:\n ", encodedOutput)

print("Encoded generated text:\n ", tokenIdsToText(encodedOutput, tokenizer))

del tokenizer, maxNewTokens, encodedOutput

Encoded output:
  tensor([[15496,   995, 16621, 37792,  9096, 29641, 33062, 32260, 37431,   815,
         38790,  8833]])
Encoded generated text:
  Hello world expressing basilheim bots dungeons productions decon shouldjri occurs


## Calculating the text generation loss

The loss is needed to backpropagate the mistakes.

We can use the cross entropy loss, which tries to minimize the difference between two distributions: the model output distribution and the input text distribution (i.e. for evaluation or training). 

To do this, we first transform the output logits of the model into a probability distribution, using the softmax function. Then, we compute the cross entropy loss.

#### Cross Entropy

The cross entropy is defined as the negative log likelihood of the probability. 
- The closer to `1` the predicted probability is, the lower the loss. 
- Likewise, the predicted ouptut closer to `0`, the higher the loss.

We provide batches of inputs to the model, thus we will compute the average cross entropy loss per batch.

#### Manual implementation
First of all, we will implement Cross Entropy manually, to better understand the concept.

1. Get logits by running model.
2. Compute probabilities as `softmax(logits)`.
3. Get `target_probabilities` corresponding to the `target_ids`.
4. Compute Cross Entropy as `-log(target_probabilities)`

In [102]:
tokenizer = tiktoken.get_encoding("gpt2")
maxNewTokens = 10
inputs = ["every effort moves", "I really like"]
targets = ["effort moves you", "really like chocolate"]

print("Inputs:\n ", inputs)

## Generate input logit tensors
input_list = [textToTokenIds(i, tokenizer) for i in inputs]
input_batch = torch.cat(input_list, dim=0)
logits = model(input_batch)
prediction = torch.argmax(logits, dim=-1)
prediction_txt = [tokenIdsToText(p, tokenizer) for p in prediction]
print("Prediction:\n ", prediction_txt)

## Generate target tensors
target_list = [textToTokenIds(i, tokenizer) for i in inputs]
target_batch = torch.cat(target_list, dim=0)
print("Target batch:\n ", target_batch)

Inputs:
  ['every effort moves', 'I really like']
Prediction:
  [' holdings awaiting meme', ' Elsa pimisition']
Target batch:
  tensor([[16833,  3626,  6100],
        [   40,  1107,   588]])


In [103]:
## Fetch probabilities for target tensors
print("Logits:\n ", logits)
probas = torch.softmax(logits, dim=-1)
print("Probas:\n ", probas)
target_probas = torch.gather(probas, -1, target_batch.unsqueeze(-1)).squeeze(-1)
print("Targer Probas:\n ", target_probas)

## Compute cross entropy
log_probas = torch.log(target_probas.flatten())
print("Log Probas:\n ", log_probas)
avg_log_probas = torch.mean(log_probas)
print("Avg. log. Probas:\n ", log_probas)
neg_avg_log_probas = torch.mean(avg_log_probas) * -1
print("Cross Entropy Loss:\n ", neg_avg_log_probas)

Logits:
  tensor([[[    -0.2121,      0.6064,     -1.1729,  ...,     -0.2882,
               0.0392,      0.4279],
         [    -1.3114,      0.4351,     -0.7188,  ...,      0.0499,
              -0.2677,      0.1474],
         [    -0.9966,      0.0661,      0.3436,  ...,     -0.1621,
              -0.0603,     -0.3128]],

        [[    -0.2090,      0.1196,     -0.3624,  ...,     -0.7420,
               0.1617,      0.1033],
         [    -0.8233,     -0.0741,     -0.6599,  ...,     -0.7223,
               0.4209,     -0.5346],
         [    -0.2380,     -0.3344,     -0.3720,  ...,      0.8198,
               0.4174,     -0.0000]]], grad_fn=<ViewBackward0>)
Probas:
  tensor([[[    0.0000,     0.0000,     0.0000,  ...,     0.0000,
              0.0000,     0.0000],
         [    0.0000,     0.0000,     0.0000,  ...,     0.0000,
              0.0000,     0.0000],
         [    0.0000,     0.0000,     0.0000,  ...,     0.0000,
              0.0000,     0.0000]],

        [[    0.0000, 

#### Compute using Torch Cross Entropy function

Now that we've implemented the cross entropy manually, we can just use the torch available functions to confirm it was correctly implemented.

In [107]:
logits_flat = logits.flatten(0,1)
targets_flat = target_batch.flatten()
torch.nn.functional.cross_entropy(logits_flat, targets_flat)

tensor(11.0994, grad_fn=<NllLossBackward0>)